In [1]:
import pandas as pd

In [2]:
BUCKET='dev-nlcd-developer'

In [3]:
df = pd.read_csv('../data/worldcities.csv')

In [4]:
df.columns

Index(['city', 'city_ascii', 'lat', 'lng', 'country', 'iso2', 'iso3',
       'admin_name', 'capital', 'population', 'id'],
      dtype='object')

In [5]:
from datetime import datetime

def now():
    # Get current date and time
    current_datetime = datetime.now()

    # Format as yyyy_mm_dd string
    formatted_date = current_datetime.strftime("%Y_%m_%d_%H_%M_%S")

    print(formatted_date)
    return formatted_date

In [6]:
nowt = now()
out=f's3://eccoe-scratch/slurp-inbox/{nowt}_work.csv'
print(out)


2023_11_21_21_26_42
s3://eccoe-scratch/slurp-inbox/2023_11_21_21_26_42_work.csv


In [7]:
#df['country'].unique()

In [8]:
fav_countries = ['United States', 'United Kingdom', 'France', 'Germany', 'Australia']

In [9]:
for c in fav_countries:
    print(c)
    df2 = df.loc[df['country'] == c]
    #print(df2.head())
    print(df2.shape)

United States
(5393, 11)
United Kingdom
(1305, 11)
France
(1140, 11)
Germany
(1733, 11)
Australia
(269, 11)


In [10]:
NUM_OF_CITIES = 0
COUNTRY = 'Australia'

In [11]:
def build_work_df(n,c):
    records=[]
    record={}
    record['country']=c
    record['quantity']=n
    records.append(record)

    df = pd.DataFrame(records)
    return df

In [12]:
def submit_work(df):
    out=f's3://{BUCKET}/slurp-inbox/{nowt}_work.csv'
    df.to_csv(out)

In [13]:
def build_work_and_quit():
    #print(NUM_OF_CITIES)
    #print(COUNTRY)
    df = build_work_df(NUM_OF_CITIES, COUNTRY)
    submit_work(df)
    print(df)
    # quit()

In [14]:
import panel as pn
pn.extension()

In [15]:
title = '# CityCo Order Form'

tif_select = pn.widgets.Select(name='fav1', options=fav_countries)

In [16]:
static_text = pn.widgets.StaticText(name='EventSeed', value='A string')

In [17]:
ls_radio = pn.widgets.RadioBoxGroup(name='num cities', options=['25','50','100'], inline=True)

In [18]:
@pn.depends(tif_select.param.value)
def refresh_image(fav1):
    global COUNTRY
    static_text = f'## {fav1}'
    COUNTRY=fav1
    return static_text

In [19]:
INITIAL=True

In [20]:
@pn.depends(ls_radio.param.value)
def refresh_and_go(numc):
    global NUM_OF_CITIES
    global INITIAL
    static_text = f'## {numc}'
    NUM_OF_CITIES = numc
    if not INITIAL:
        build_work_and_quit()
    else:
        INITIAL = False
    return static_text

In [21]:
b = pn.Column(pn.Row(title, tif_select,ls_radio),
              pn.Row(refresh_image),
              pn.Row(refresh_and_go))

In [22]:
b.servable()

country quantity 
0 France 50 
 country quantity 
0 Germany 100 
 country quantity 
0 France 25

Column
    [0] Row
        [0] Markdown(str)
        [1] Select(name='fav1', options=['United States', ...], value='United States')
        [2] RadioBoxGroup(inline=True, name='num cities', options=['25', '50', '100'], value='25')
    [1] Row
        [0] ParamFunction(function, _pane=Markdown, defer_load=False)
    [2] Row
        [0] ParamFunction(function, _pane=Markdown, defer_load=False)